In [ ]:
load("parameters.py")

In [ ]:
import hashlib
# Ex. 1
# N = p*q*r

# Using hint 1, find one of the prime factors to be the gcd among N_i and N
p = 1
for idx1 in range(len(Q1_L_mod)):
    for idx2 in range(len(Q1_L_mod)):
        if idx1 < idx2:
            mod_gcd = gcd(Q1_L_mod[idx1], Q1_L_mod[idx2])
            if mod_gcd != 1 and is_prime(mod_gcd):
                p = mod_gcd
    if gcd(Q1_L_mod[idx1], Q1_N) != 1 and is_prime(gcd(Q1_L_mod[idx1], Q1_N)):
        p = gcd(Q1_L_mod[idx1], Q1_N)

# Find the pair of plaintext + wrong decryption
wrong_pt_decrypted = []
for pair in Q1_L_x:
    if pair[0] != pair[1]:
        wrong_pt_decrypted = pair

pt = wrong_pt_decrypted[0]
decryption = wrong_pt_decrypted[1]
# Using fault-attack to find another prime factor
q = gcd(decryption - pt, Q1_N//p)
# Calculate the last prime factor
r = Q1_N//(p*q)

# print("p = " + str(p) + " : " + str(is_prime(p)))
# print("q = " + str(q) + " : " + str(is_prime(q)))
# print("r = " + str(r) + " : " + str(is_prime(r)))

# Find d
phi = (p-1)*(q-1)*(r-1)
d = inverse_mod(Q1_e, phi)
# print("d = " + str(d))

# reverse RSA
decrypted_rsa = power_mod(Q1_ct_RSA, d, Q1_N)
# NOTE: This didn't work in sage??
# key = hashlib.sha512(str(decrypted_rsa).encode('ascii')).digest()
# ct_m = bytes(a ^^ b for (a, b) in zip(key, blah))
# print(ct_m)

# Below doesn't work here either but does run in a python 3 notebook and worked (password is 'cults35')
# TODO: need to fix my sage's python version
# key = hashlib.sha512(str(decrypted_rsa).encode('ascii')).digest()
# ct_m = bytes(a ^ b for (a, b) in zip(key, Q1_ct_pwd))
# print(ct_m)

In [ ]:
# Ex.2
import collections
exponents = map(lambda x: x[1], Q2_G) 
occurrences = collections.Counter(exponents).most_common()

# most common exponent is 271
most_common_exponent = occurrences[0][0]
# it occurrs 271 times
exponent_occurrences = occurrences[0][1]

# extract the moduli and ciphers corresponding to the most common exponent
relevant_indices = []
relevant_moduli = []
relevant_ciphers = []
for idx in range(len(Q2_G)):
    if Q2_G[idx][1] == most_common_exponent:
        relevant_indices.append(idx)
        relevant_moduli.append(Q2_G[idx][0])
        relevant_ciphers.append(Q2_ciphertexts[idx])

# In order to use CRT, we want all possible pairs of moduli to be coprime
all_coprime = True
for idx_one in range(len(relevant_moduli)):
    for idx_two in range(len(relevant_moduli)):
        if (idx_one < idx_two):
            if gcd(relevant_moduli[idx_one], relevant_moduli[idx_two]) != 1:
                all_coprime = False
                
if all_coprime == False:
    print("Too bad... we cannot use the CRT!!")
else:
    print("All moduli are coprime!! Let's use CRT :)")

# CRT
# http://rosettacode.org/wiki/Chinese_remainder_theorem#Python_3.6
moduli_product = prod(relevant_moduli)
modular_sum = mod(0, moduli_product)
for idx in range(len(relevant_moduli)):
    curr_modulus = relevant_moduli[idx]
    temp_product = moduli_product // curr_modulus
    modular_sum = modular_sum + (relevant_ciphers[idx] * temp_product * inverse_mod(temp_product, curr_modulus))

lifted = modular_sum.lift()
pt = lifted.nth_root(most_common_exponent)
print(pt)

In [ ]:
# Ex.3
import binascii
# Given two polynomials, find gcd using euclidean
def find_gcd(g1,g2):
    if g2 == 0:
        return g1.monic()
    else:
        return find_gcd(g2, g1%g2)

#http://crypto.stanford.edu/~dabo/pubs/papers/RSA-survey.pdf
def coppersmith(c1, c2, N, e):
    # Coppersmith's padding implements section 4.4, Theorem 8 from the reference above
    # create polynomials and change their base ring   
    R = Zmod(N)
    P1.<x,y> = PolynomialRing(R)
    P2.<y> = PolynomialRing(R)
    g1 = (x^e - c1).change_ring(P2)
    g2 = ((x + y)^e - c2).change_ring(P2)
    
    # h(y) = res_x(g1,g2)
    h = g1.resultant(g2, variable=x)
    #print(type(h))
    # convert the resultant to univariate polynomial with coefficients mod N
    univariate_h = h.univariate_polynomial().change_ring(R)
    #print(type(univariate_res_x))

    #https://doc.sagemath.org/html/en/reference/polynomial_rings/sage/rings/polynomial/polynomial_modn_dense_ntl.html\#sage.rings.polynomial.polynomial_modn_dense_ntl.small_roots
    # empirically found beta=1.0 and epsilon=1/10 to be giving results
    beta = 1.0
    epsilon = 1/10
    roots = univariate_h.small_roots(beta=beta, epsilon=epsilon)
    #print(roots)
    # delta --> delta between the two plaintexts m1, m2
    delta = roots[0]

    # Franklin-Reiter related messages attack implements section 4.3, Lemma 7
    if e != 3:
        print("Ooops, we want e=3!")
    else:
        # create polynomials
        P.<x> = PolynomialRing(R)
        # we use the delta found in the coppersmith's attack
        g1 = x^e - c1
        g2 = (x + delta)^e - c2
        # Euclidean algorithm to find gcd of g1,g2
        recovered = -find_gcd(g1, g2).coefficients()[0]

        # M = recovered*2^i
        for i in range(pow(2,Q3b_e)):
            msg = hex(Integer(recovered*pow(2,i)))
#             print(msg)
            # string must be even-length
            if(len(msg)%2 == 1):
                msg = '0' + msg
            print((binascii.unhexlify(msg)) + "\n")
coppersmith(Q3b_c1, Q3b_c2, Q3b_N, Q3b_e)